In [2]:
import urllib
from urllib.error import HTTPError
import requests
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import csv
from xml import etree
from xml.sax.saxutils import unescape
import xml.etree.ElementTree as ET
import json
import html
import pandas as pd
import numpy as np
import re
import unicodedata
import datetime

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
def get_root(url):
    req = urllib.request.Request(url=url)
    try:
        handler = urllib.request.urlopen(req)
        tree = ET.ElementTree(file = handler)
        root = tree.getroot()
        return root
    except HTTPError as e:
        content = e.read()
        return content

In [5]:
base_url = "https://www.costa-coffee.cz/clanky.php"

In [6]:
root = get_root(base_url)  

In [7]:
def get_column_names(root):
    tags = []
    for i, child in enumerate(root):  
        for subchild in child[-1:]: 
            for k in subchild:
                tags.append(k.tag)
    return tags 

In [8]:
tags = get_column_names(root)

In [9]:
def get_data(root):
    records = []
    data = []
    for i, child in enumerate(root):
        for subchild in child:
            for j in subchild:
                records.append(j.text)
            try:
                if len(records) != 0:
                    data.append(records)
                    records = []
            except AttributeError as error:
                return error
    return data

In [10]:
records = get_data(root)

In [11]:
rawDf = pd.DataFrame(records, columns=tags)

In [12]:
articleDf = rawDf[["title", "description", "pubDate", "image"]]

In [13]:
def unescape(x):
        if x != None:
            return (html.unescape(x).encode("utf-8", "xmlcharrefreplace").decode())

In [14]:
def sanitize_html(x):
    """Returns the given HTML with all tags stripped."""
    if x != None:
        x = x.rstrip()
        return re.sub(r'<[^>]*?>', '', x)

In [15]:
def update_unicode(x):
    if x != None:
        x.strip()
        x.replace('\xa0', ' ')
        x.replace('\xad', ' ')
        return unicodedata.normalize("NFKD", x)

In [16]:
def update_date(x):
    return datetime.datetime.strptime(x, '%a, %d %b %Y %H:%M:%S %z').strftime('%d. %m. %Y %H:%M:%S')    

In [17]:
articleCleanDf = articleDf.copy()

In [18]:
articleDf.loc[:,('description')] = articleDf.loc[:,('description')].apply(unescape).apply(sanitize_html).apply(update_unicode)

In [19]:
articleDf.loc[:,('title')] = articleDf.loc[:,('title')].apply(unescape).apply(sanitize_html).apply(update_unicode)

In [20]:
articleDf.loc[:,('pubDate')] = articleDf.loc[:,('pubDate')].apply(update_date)

In [21]:
articleDf = articleDf.replace([None], "")

In [ ]:
articleDf

In [148]:
def df_to_json(df):
    article_json = df.to_json(orient='records')
    temp_po = json.loads(article_json, encoding='utf-8')
    temp_json = json.dumps(temp_po, ensure_ascii = False)
    json_no_space = temp_json.replace('\xad', ' ')
    json_no_space = temp_json.replace('\\', '')
    json_no_space = temp_json.replace('\n', ' ')
    return json_no_space

In [149]:
b = df_to_json(articleDf)

In [150]:
def print_json(json):
    return print(json)

In [ ]:
print_json(b)